In [4]:
import pandas as pd
import numpy as np
from haversine import haversine
import networkx as nx

df = pd.read_csv('4h_timeFrame_not_at_3_fing_am.csv')

In [181]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
df.head()

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom,velocity,datetime
0,1.884809e+07,227408710.0,0.0,0.0,0.000000,258.100000,511.000000,-4.490398,48.379186,1.456818e+09,POINT (-4.4903984 48.3791865),0.013741,2016-03-01 07:36:00
1,1.758572e+07,228853000.0,7.0,-127.0,6.169685,200.775984,511.000000,-3.891684,47.732670,1.456818e+09,POINT (-3.891684385826772 47.73266998031496),0.153449,2016-03-01 07:36:00
2,1.884808e+07,227686540.0,0.0,0.0,6.400000,199.300000,511.000000,-4.464395,48.327780,1.456818e+09,POINT (-4.464395 48.32778),6.911790,2016-03-01 07:36:00
3,1.751793e+07,226216000.0,7.0,0.0,0.000000,196.639167,302.466667,-4.175265,47.835712,1.456818e+09,POINT (-4.175265398333333 47.83571188333333),0.001396,2016-03-01 07:36:00
4,1.751790e+07,227270000.0,5.0,-127.0,0.000000,0.000000,511.000000,-4.478274,48.383153,1.456818e+09,POINT (-4.478274466666667 48.38315333333333),0.003177,2016-03-01 07:36:00


In [16]:
def pairs_in_radius(df, diam=1000):
    res = []
    for ind_i, ind_j, val_i, val_j in nparray_combinations(df):
        dist = haversine(val_i, val_j)*1000
        if (dist<diam):
            res.append((ind_i,ind_j))   
    return res
            

In [165]:
def connected_edges(data):
    G = nx.Graph()
    G.add_edges_from(data)
    return [sorted(list(cluster)) for cluster in nx.find_cliques(G)]

In [8]:
def nparray_combinations(arr):
    for i in range(arr.shape[0]):
        for j in range(i+1, arr.shape[0]):
            yield i, j, arr[i,:], arr[j,:]

In [9]:
def eval_candidate(candidate, pairs):
    for pair in itertools.combinations(candidate,2):
        if pair not in pairs:
            return False
    return True

In [10]:
def translate(sets, sdf):
    return [tuple([sdf.iloc[point].mmsi for point in points]) for points in sets]

In [11]:
def circle_cluster(timeframe, min_cardinality = 2):
    pairs = pairs_in_radius(timeframe[['lon', 'lat']].values)
    full_sets = connected_edges(pairs)
    clusters = []
    for individual_sets in full_sets:
        # MAYBE FROM BIG TO SMALL INSTEAD OF SMALL TO BIG
        for j in range(min_cardinality,len(individual_sets)+1):
            for candidate in itertools.combinations(individual_sets,j):
                 if eval_candidate(candidate, pairs):
                        clusters.append(candidate)
    return clusters

In [12]:
def get_circular_clusters(timeframe):
    pairs = pairs_in_radius(timeframe[['lon', 'lat']].values)
    return connected_edges(pairs)
    

In [186]:
keep = pd.DataFrame()
log=[]
for ind, (ts, sdf) in enumerate(df.groupby('datetime')):
    print (ind+1,' / ', df.datetime.nunique(), end='\r')
    present = pd.DataFrame([[tuple(val)] for (val) in translate(get_circular_clusters(sdf), sdf)], columns=['clusters'])
    present['st'] = present['et'] = ts
    present['dur'] = 1
    if ind == 0:
        past = present
        continue
    
    past
    new = present.merge(past, how='left', indicator=True)
    new = new[new['_merge']=='left_only'].drop(['_merge'],axis=1)
    new
    
#     new['st'] = new['et'] = ts
#     new['dur'] = 1

#     get if tuple of tmp1 is subset or equal of a row of tmp2
    tmp = pd.DataFrame(past['clusters']).apply(lambda x: True in [set(x) <= set(val) for val in present.values], axis=1)
    tst = past.loc[tmp].copy()
    tst['et'] = ts
    tst['dur'] = tst.dur.apply(lambda x : x+1)
    
    past = pd.concat([tst,new]).reset_index(drop=True)
    
    keep = keep.append(past.loc[past.dur>=5])
    log.append(past.dur)
#     if ind==2: break
#     break
    if ind==50:
        break
# fn_keep = keep.drop_duplicates(['clusters'], keep='last')

,clusters,st,et,dur
0,"(227686540.0, 227632830.0)",2016-03-01 07:36:00,2016-03-01 07:36:00,1
1,"(227686540.0, 227322670.0)",2016-03-01 07:36:00,2016-03-01 07:36:00,1
2,"(227270000.0, 227008170.0, 227002330.0, 226263...",2016-03-01 07:36:00,2016-03-01 07:36:00,1
3,"(227270000.0, 227222000.0, 227002330.0, 227016...",2016-03-01 07:36:00,2016-03-01 07:36:00,1
4,"(249297000.0, 477115900.0)",2016-03-01 07:36:00,2016-03-01 07:36:00,1
5,"(227142200.0, 227941000.0, 227162950.0)",2016-03-01 07:36:00,2016-03-01 07:36:00,1
6,"(227632830.0, 227303430.0)",2016-03-01 07:36:00,2016-03-01 07:36:00,1
7,"(227639660.0, 227611930.0, 227650230.0, 227578...",2016-03-01 07:36:00,2016-03-01 07:36:00,1
8,"(227639660.0, 227611930.0, 227322670.0, 227650...",2016-03-01 07:36:00,2016-03-01 07:36:00,1
9,"(227590030.0, 227588970.0)",2016-03-01 07:36:00,2016-03-01 07:36:00,1


,clusters,st,et,dur
0,"(227006750.0, 227270000.0, 227002330.0, 226263...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
1,"(227006750.0, 227270000.0, 227002330.0, 227222...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
2,"(227578460.0, 227650230.0, 227322670.0, 227639...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
3,"(227578460.0, 227650230.0, 227631450.0, 227639...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
4,"(227322670.0, 227686540.0, 227639660.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
5,"(227162950.0, 227142200.0, 227941000.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
6,"(227640710.0, 227654220.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
7,"(227315110.0, 227666970.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
8,"(227686540.0, 227641920.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
9,"(227590030.0, 227588970.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1


,clusters,st,et,dur
0,"(227632830.0, 227303430.0)",2016-03-01 07:36:00,2016-03-01 07:37:00,2
1,"(227590030.0, 227588970.0)",2016-03-01 07:36:00,2016-03-01 07:37:00,2
2,"(227641920.0, 636015106.0)",2016-03-01 07:36:00,2016-03-01 07:37:00,2
3,"(228064900.0, 228186700.0)",2016-03-01 07:36:00,2016-03-01 07:37:00,2
4,"(227006750.0, 227270000.0, 227002330.0, 226263...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
5,"(227006750.0, 227270000.0, 227002330.0, 227222...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
6,"(227578460.0, 227650230.0, 227322670.0, 227639...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
7,"(227578460.0, 227650230.0, 227631450.0, 227639...",2016-03-01 07:37:00,2016-03-01 07:37:00,1
8,"(227322670.0, 227686540.0, 227639660.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1
9,"(227162950.0, 227142200.0, 227941000.0)",2016-03-01 07:37:00,2016-03-01 07:37:00,1


,clusters,st,et,dur
0,"(227142200.0, 227162950.0, 227941000.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
1,"(227632830.0, 227303430.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
2,"(227003050.0, 228064900.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
3,"(227654220.0, 227640710.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
4,"(227639660.0, 227686540.0, 227650230.0, 227322...",2016-03-01 07:38:00,2016-03-01 07:38:00,1
5,"(227639660.0, 227578460.0, 227631450.0, 227650...",2016-03-01 07:38:00,2016-03-01 07:38:00,1
6,"(477115900.0, 249297000.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
7,"(227589520.0, 227322670.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
8,"(228186700.0, 228064900.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
9,"(227315110.0, 227666970.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1


,clusters,st,et,dur
0,"(227632830.0, 227303430.0)",2016-03-01 07:36:00,2016-03-01 07:38:00,3
1,"(227590030.0, 227588970.0)",2016-03-01 07:36:00,2016-03-01 07:38:00,3
2,"(227641920.0, 636015106.0)",2016-03-01 07:36:00,2016-03-01 07:38:00,3
3,"(227315110.0, 227666970.0)",2016-03-01 07:37:00,2016-03-01 07:38:00,2
4,"(227590030.0, 227588970.0)",2016-03-01 07:37:00,2016-03-01 07:38:00,2
5,"(227641920.0, 636015106.0)",2016-03-01 07:37:00,2016-03-01 07:38:00,2
6,"(227632830.0, 227303430.0)",2016-03-01 07:37:00,2016-03-01 07:38:00,2
7,"(477115900.0, 249297000.0)",2016-03-01 07:37:00,2016-03-01 07:38:00,2
8,"(227142200.0, 227162950.0, 227941000.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1
9,"(227632830.0, 227303430.0)",2016-03-01 07:38:00,2016-03-01 07:38:00,1


...

,clusters,st,et,dur
0,"(227016100.0, 227270000.0, 227222000.0, 227006...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
1,"(227016100.0, 227008170.0, 227270000.0, 227006...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
2,"(227666970.0, 227621940.0, 227315110.0, 227113...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
3,"(227650230.0, 227590030.0, 227641920.0, 227322...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
4,"(227650230.0, 227641920.0, 227611930.0, 227569...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
5,"(227632830.0, 227588970.0, 227589520.0, 227303...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
6,"(227142200.0, 227941000.0, 227162950.0)",2016-03-01 08:24:00,2016-03-01 08:24:00,1
7,"(227640710.0, 227631450.0, 227322670.0, 227690...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
8,"(477115900.0, 249297000.0)",2016-03-01 08:24:00,2016-03-01 08:24:00,1
9,"(227590030.0, 227641920.0, 227322670.0, 227690...",2016-03-01 08:24:00,2016-03-01 08:24:00,1


,clusters,st,et,dur
0,"(477115900.0, 249297000.0)",2016-03-01 08:22:00,2016-03-01 08:24:00,3
1,"(477115900.0, 249297000.0)",2016-03-01 08:23:00,2016-03-01 08:24:00,2
2,"(228064900.0, 228186700.0)",2016-03-01 08:23:00,2016-03-01 08:24:00,2
3,"(227142200.0, 227941000.0, 227162950.0)",2016-03-01 08:23:00,2016-03-01 08:24:00,2
4,"(227762340.0, 227636480.0)",2016-03-01 08:23:00,2016-03-01 08:24:00,2
5,"(227016100.0, 227270000.0, 227222000.0, 227006...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
6,"(227016100.0, 227008170.0, 227270000.0, 227006...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
7,"(227666970.0, 227621940.0, 227315110.0, 227113...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
8,"(227650230.0, 227590030.0, 227641920.0, 227322...",2016-03-01 08:24:00,2016-03-01 08:24:00,1
9,"(227650230.0, 227641920.0, 227611930.0, 227569...",2016-03-01 08:24:00,2016-03-01 08:24:00,1


,clusters,st,et,dur
0,"(227632830.0, 227654220.0, 227303430.0, 227588...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
1,"(227632830.0, 227303430.0, 227589520.0, 227588...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
2,"(249297000.0, 477115900.0)",2016-03-01 08:25:00,2016-03-01 08:25:00,1
3,"(227640710.0, 227690560.0, 227322670.0, 227631...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
4,"(227578460.0, 227650230.0, 227690560.0, 227569...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
5,"(227578460.0, 227690560.0, 227569380.0, 227590...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
6,"(227578460.0, 227650230.0, 227569380.0, 227611...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
7,"(227636480.0, 227762340.0)",2016-03-01 08:25:00,2016-03-01 08:25:00,1
8,"(227113100.0, 227640190.0)",2016-03-01 08:25:00,2016-03-01 08:25:00,1
9,"(227113100.0, 227666970.0, 227315110.0, 227621...",2016-03-01 08:25:00,2016-03-01 08:25:00,1


,clusters,st,et,dur
0,"(228064900.0, 228186700.0)",2016-03-01 08:23:00,2016-03-01 08:25:00,3
1,"(228064900.0, 228186700.0)",2016-03-01 08:24:00,2016-03-01 08:25:00,2
2,"(227632830.0, 227654220.0, 227303430.0, 227588...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
3,"(227632830.0, 227303430.0, 227589520.0, 227588...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
4,"(249297000.0, 477115900.0)",2016-03-01 08:25:00,2016-03-01 08:25:00,1
5,"(227640710.0, 227690560.0, 227322670.0, 227631...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
6,"(227578460.0, 227650230.0, 227690560.0, 227569...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
7,"(227578460.0, 227690560.0, 227569380.0, 227590...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
8,"(227578460.0, 227650230.0, 227569380.0, 227611...",2016-03-01 08:25:00,2016-03-01 08:25:00,1
9,"(227636480.0, 227762340.0)",2016-03-01 08:25:00,2016-03-01 08:25:00,1


,clusters,st,et,dur
0,"(227650230.0, 227690560.0, 227639660.0, 227578...",2016-03-01 08:26:00,2016-03-01 08:26:00,1
1,"(227631450.0, 227690560.0, 227639660.0, 227686...",2016-03-01 08:26:00,2016-03-01 08:26:00,1
2,"(227631450.0, 227690560.0, 227639660.0, 227640...",2016-03-01 08:26:00,2016-03-01 08:26:00,1
3,"(227690560.0, 227639660.0, 227322690.0, 227686...",2016-03-01 08:26:00,2016-03-01 08:26:00,1
4,"(227016100.0, 227362110.0, 227369960.0, 227006...",2016-03-01 08:26:00,2016-03-01 08:26:00,1
5,"(227362110.0, 227369960.0, 227806500.0, 227592...",2016-03-01 08:26:00,2016-03-01 08:26:00,1
6,"(227362110.0, 227369960.0, 227008170.0, 227592...",2016-03-01 08:26:00,2016-03-01 08:26:00,1
7,"(228796000.0, 227362110.0, 227369960.0, 227806...",2016-03-01 08:26:00,2016-03-01 08:26:00,1
8,"(227362110.0, 227369960.0, 227006750.0, 227008...",2016-03-01 08:26:00,2016-03-01 08:26:00,1
9,"(228796000.0, 227362110.0, 227369960.0, 227806...",2016-03-01 08:26:00,2016-03-01 08:26:00,1


In [180]:
keep

,clusters,st,et,dur
0,"(477115900.0, 249297000.0)",2016-03-01 07:45:00,2016-03-01 07:49:00,5
0,"(477115900.0, 249297000.0)",2016-03-01 07:45:00,2016-03-01 07:50:00,6
1,"(477115900.0, 249297000.0)",2016-03-01 07:46:00,2016-03-01 07:50:00,5
0,"(249297000.0, 477115900.0)",2016-03-01 08:17:00,2016-03-01 08:21:00,5


In [162]:
keep

""


In [159]:
for i in (range(50)):
    print(log[i].max())

2
1
2
2
3
2
2
2
1
2
2
2
2
2
2
2
2
2
2
3
1
1
2
1
1
2
1
2
3
3
2
3
4
5
2
2
2
2
2
3
2
2
3
4
2
2
2
2
2
2


In [40]:
new['dur'] = new.dur.apply(lambda x : x+1)

In [44]:
past.loc[tmp]

,0
18,"(227270000.0, 227222000.0, 227016100.0)"
21,"(227270000.0, 227008170.0, 227016100.0)"
22,"(227270000.0, 227002330.0, 227016100.0)"
23,"(227270000.0, 226263000.0, 227016100.0)"
80,"(227408710.0, 227008170.0, 227369960.0)"


In [17]:
pairs = pairs_in_radius(sdf[['lon', 'lat']].values)
full_sets = connected_edges(pairs)
#     clusters = []
#     for individual_sets in full_sets:
#         # MAYBE FROM BIG TO SMALL INSTEAD OF SMALL TO BIG
#         for j in range(min_cardinality,len(individual_sets)+1):
#             for candidate in itertools.combinations(individual_sets,j):
#                  if eval_candidate(candidate, pairs):
#                         clusters.append(candidate)
#     return clusters

In [18]:
full_sets

[[0, 13, 64],
 [0, 15, 64],
 [0, 16, 64],
 [0, 24, 64],
 [0, 26, 64],
 [0, 27, 64],
 [0, 29, 64],
 [0, 32, 64],
 [0, 44, 64],
 [0, 45, 64],
 [0, 61, 64],
 [13, 41, 64],
 [16, 41, 64],
 [24, 41, 64],
 [26, 41, 64],
 [32, 41, 64],
 [41, 45, 64],
 [41, 61, 64],
 [4, 7, 41],
 [7, 16, 41],
 [7, 41, 45],
 [4, 13, 41],
 [4, 16, 41],
 [4, 24, 41],
 [4, 41, 45],
 [0, 13, 61],
 [0, 15, 61],
 [0, 16, 61],
 [0, 22, 61],
 [0, 24, 61],
 [0, 26, 61],
 [0, 27, 61],
 [0, 29, 61],
 [0, 32, 61],
 [0, 44, 61],
 [0, 45, 61],
 [0, 15, 50],
 [0, 22, 50],
 [0, 23, 50],
 [0, 26, 50],
 [0, 27, 50],
 [0, 32, 50],
 [0, 44, 50],
 [0, 47, 50],
 [0, 15, 47],
 [0, 22, 47],
 [0, 23, 47],
 [0, 27, 47],
 [0, 32, 47],
 [0, 44, 47],
 [0, 13, 45],
 [0, 16, 45],
 [0, 24, 45],
 [0, 26, 45],
 [0, 32, 45],
 [0, 15, 44],
 [0, 22, 44],
 [0, 23, 44],
 [0, 26, 44],
 [0, 27, 44],
 [0, 32, 44],
 [0, 13, 32],
 [0, 15, 32],
 [0, 16, 32],
 [0, 22, 32],
 [0, 23, 32],
 [0, 24, 32],
 [0, 26, 32],
 [0, 27, 32],
 [0, 29, 32],
 [0, 13, 29],


In [69]:
[(227006750.0, 227008170.0, 227003050.0)] in pd.DataFrame([[val] for (val) in translate(full_sets, sdf)]).values.tolist()

True

In [21]:
pd.DataFrame([[tuple(val)] for (val) in translate(full_sets, sdf)])

,0
0,"(227574020.0, 228394000.0, 227806500.0)"
1,"(227574020.0, 227640710.0, 227806500.0)"
2,"(227574020.0, 227311000.0, 227806500.0)"
3,"(227574020.0, 228236600.0, 227806500.0)"
4,"(227574020.0, 227705102.0, 227806500.0)"
5,"(227574020.0, 227666970.0, 227806500.0)"
6,"(227574020.0, 256494000.0, 227806500.0)"
7,"(227574020.0, 228211900.0, 227806500.0)"
8,"(227574020.0, 227632830.0, 227806500.0)"
9,"(227574020.0, 227303430.0, 227806500.0)"


In [92]:
tmp1 = pd.DataFrame([[val] for val in translate(full_sets, sdf)])

In [89]:
tmp2 = pd.DataFrame([[val] for val in translate(full_sets, sdf)])

In [127]:
tmp2 = tmp2.append( pd.DataFrame([[(227322690.0, 228762000.0, 228893000.0, 228893000.0)]] )).reset_index(drop=True)

In [143]:
[tmp1.iloc[0][0] in val[0] for val in tmp2.values.tolist()]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [142]:
tmp1.iloc[0][0]

(227322690.0, 228762000.0, 228893000.0)

In [118]:
tmp2 = tmp2.append([[(234056000.0, 227632830.0, 227639660.0, 237639660.0)]])

In [111]:
tmp2.merge(tmp1, how='left')

,0
0,"(234056000.0, 227632830.0, 227639660.0)"
1,"(234056000.0, 227640190.0, 227639660.0)"
2,"(227612860.0, 228394000.0, 227162950.0)"
3,"(227016100.0, 227142200.0, 227654220.0)"
4,"(227322690.0, 228762000.0, 228893000.0)"


In [112]:
tmp2.loc[~tmp2.isin(tmp1)]

ValueError: Cannot index with multidimensional key

In [1]:
new = present.merge(past, how='left', indicator=True)
new[new['_merge']=='left_only'].drop(['_merge'],axis=1)

NameError: name 'present' is not defined

In [2]:
# get if tuple of tmp1 is subset or equal of a row of tmp2
past.apply(lambda x: True in [set(x) <= set(val) for val in present.values], axis=1)

NameError: name 'past' is not defined

In [148]:
tmp1

,0
0,"(227322690.0, 228762000.0, 228893000.0)"
1,"(227322690.0, 220364000.0, 228893000.0)"
2,"(477115900.0, 227300000.0, 256494000.0)"
3,"(228064900.0, 228796000.0, 227611930.0)"


In [149]:
tmp2

,0
0,"(234056000.0, 227632830.0, 227639660.0)"
1,"(234056000.0, 227640190.0, 227639660.0)"
2,"(227612860.0, 228394000.0, 227162950.0)"
3,"(227016100.0, 227142200.0, 227654220.0)"
4,"(227322690.0, 228762000.0, 228893000.0)"
5,"(234056000.0, 227632830.0, 227639660.0, 237639..."
6,"(227322690.0, 228762000.0, 228893000.0, 228893..."


In [167]:
[set(val) for val in tmp2.values]

[{(234056000.0, 227632830.0, 227639660.0)},
 {(234056000.0, 227640190.0, 227639660.0)},
 {(227612860.0, 228394000.0, 227162950.0)},
 {(227016100.0, 227142200.0, 227654220.0)},
 {(227322690.0, 228762000.0, 228893000.0)},
 {(234056000.0, 227632830.0, 227639660.0, 237639660.0)},
 {(227322690.0, 228762000.0, 228893000.0, 228893000.0)}]

In [166]:
tmp2

,0
0,"(234056000.0, 227632830.0, 227639660.0)"
1,"(234056000.0, 227640190.0, 227639660.0)"
2,"(227612860.0, 228394000.0, 227162950.0)"
3,"(227016100.0, 227142200.0, 227654220.0)"
4,"(227322690.0, 228762000.0, 228893000.0)"
5,"(234056000.0, 227632830.0, 227639660.0, 237639..."
6,"(227322690.0, 228762000.0, 228893000.0, 228893..."
